In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from datetime import datetime as dt
import re
import MeCab
import ipadic
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import scipy as sp
import pickle

In [2]:
with open('/app/patent/df.pkl', 'rb') as f:
    df = pickle.load(f)

In [3]:
df.head()

,出願年,出願人・権利者名,IPC,Ｆターム,要約,詳細な説明
0,2001,佐藤工業株式会社,B65D,"3E060AA03,3E060AB05,3E060BC02,3E060CB04,3E060C...",【要約】【課題】 強度が劣る原紙を用いたとしても箱の強度が維持される外装用段ボール箱を得んと...,【考案の詳細な説明】 【０００１】 【考案の属する技術分野】 本考案は、袋菓子または詰...
1,2001,"寄神建設株式会社,東亜建設工業株式会社,社団法人日本海上起重技術協会",B66C,"3F004CC01,3F004CD08",【要約】【課題】 重量物を吊り下げて設置を行う際の、重量物の開放を遠隔操作で容易かつ安全に行...,【考案の詳細な説明】【０００１】【考案の属する技術分野】 本考案は、クレーン等によりコンクリ...
2,2001,株式会社大林組,E01D,"2D059AA03,2D059CC04",【要約】【目的】 作業足場や型枠の組立，解体が不要になる橋脚柱の構築工法の提供。【構成】 構...,【発明の詳細な説明】【０００１】【産業上の利用分野】この発明は、橋脚柱の構築工法および橋脚柱...
3,2001,株式会社大林組,"E02B,B65G","3F040BA01,3F040CA04,3F040EA03",【要約】【課題】 材料分離と運搬費用の低減。【解決手段】 シュート式打設装置１２は、円筒状の...,【発明の詳細な説明】【０００１】【発明の属する技術分野】この発明は、コンクリートの運搬装置に...
4,2001,株式会社大林組,E02B,NaN,【要約】【課題】 材料分離と運搬費用の低減。【解決手段】 シュート式打設装置１２は、円筒状の...,【発明の詳細な説明】【０００１】【発明の属する技術分野】この発明は、コンクリートの運搬方法に...


In [4]:
df['all'] = df['要約']+df['詳細な説明']

In [5]:
excepword = ["位置","部","上記","上部","上端","下記","下部","下端","両端","内部","外部","前記","複数","横","図","要約","方法","手段","対象","現場","評価" ,"可能","構成","当該","以下","以上","該当","項","各","基準","端部","請求", "特許","本","一定","同上", '方向', '程度']  #除外する特定の名詞
exceptype= ["接尾","副詞可能","数詞","助数詞","代名詞","複数"]  #除外する名詞のタイプ

def tokenize_rm_setsubi(sentence):
    """ 連名詞のリストをトークンとして返す """
    print(".",end='') #動いてるのか不安になるからプリント
    sentence = re.sub('\d+', '', sentence)   # 数字を削除(macabでなぜか名詞に認定された数字あったからここで)
    sentence = re.sub(r'[ -/:-@\[-~]', "。", sentence)#半角記号,数字,英字
    sentence = re.sub(r'[！-／：-＠［-｀｛-～、-〜”’%]+/g', "。", sentence)#全角記号
    sentence = re.sub(r'[[０-９ａ-ｚＡ-Ｚ]]', '', sentence) #全角英数を削除
    sentence = re.sub(r'【.+】+', '', sentence) #全角英数を削除
    
    
    wakati = MeCab.Tagger(ipadic.MECAB_ARGS)
    node = wakati.parseToNode(sentence)
    sequence = 2   #何連名詞以上をかえすか
    dictionary = []
    prev_seq = 0  #その前にいくつ名詞続いたか
    flag = 0  #
    while node:
        word = node.surface
        hinshi = node.feature.split(",")

        if prev_seq  : # その前に名詞が1以上つづいていたら
            if  (hinshi[0] == "名詞" ) and  (hinshi[0] not in exceptype) and  (hinshi[1] not in exceptype )and (hinshi[2] not in exceptype )and (word not in excepword) : #今回、除外名詞以外だったら、
                dictionary[-1] = dictionary[-1]+word   # 辞書の最後の単語にこの単語を結合
                prev_seq  +=1
                flag = 0
                
            elif (hinshi[1] == "接尾" ): #接尾辞だったら
                dictionary[-1] = dictionary[-1]+word   # とりあえず辞書の最後の単語にこの単語を結合して
                prev_seq  +=1
                flag = 1 #フラグを1に

            elif flag == 1: # 一個前が接尾辞で今度が名詞でなかったら、一個前の接尾辞で終わってる単語を辞書から削除
                dictionary.pop(-1)   #一個前の接尾辞で終わってる単語を辞書から削除
                prev_seq  =0
                flag = 0
            else:
                flag = 0
                if prev_seq  < sequence: 
                    dictionary.pop(-1) #一個前の単語を辞書から削除
                prev_seq = 0
        else:  # その前に名詞がつづいてなくて
            if  (hinshi[0] == "名詞" or hinshi[0] == "接頭辞")  and  hinshi[0] not in exceptype and  hinshi[1] not in exceptype and hinshi[2] not in exceptype and word not in excepword :
                dictionary.append(word)
                prev_seq += 1
                flag = 0

        node = node.next  
    return dictionary



# 単語ー文書行列の作成
docs = np.array(df['all'])

tfidf_vectorizer = TfidfVectorizer(
    tokenizer = tokenize_rm_setsubi, #上でつくった関数
    lowercase=True,
    max_df=5000,
    # smooth_idf = False,
    min_df=5)

tfidf_vectorizer.fit(docs)
terms= tfidf_vectorizer.get_feature_names()

count_vectrizer = CountVectorizer(
    vocabulary = terms,
    tokenizer = tokenize_rm_setsubi, #上でつくった関数
)



/opt/conda/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/tmp/ipykernel_139/4217999940.py:10: FutureWarning: Possible nested set at position 1
  sentence = re.sub(r'[[０-９ａ-ｚＡ-Ｚ]]', '', sentence) #全角英数を削除


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [6]:
len(terms)

8247

In [7]:
c_matrix = count_vectrizer.transform(docs)
sp.sparse.save_npz('c_matrix_allterm',c_matrix)
terms_df = pd.DataFrame(terms)
terms_df.to_csv('terms_df.csv', mode='w')

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [8]:
c_df = pd.DataFrame(terms_df.values, columns=["terms"] )
c_df["document_frequency"] =  np.array(c_matrix.astype(bool).sum(axis=0).reshape(-1,1))
c_df["max_term_frequency"]  = c_matrix.T.max(axis=1).toarray()
c_df["sumof_term_frequency"] = np.array(c_matrix.T.sum(axis=1))
c_df.sort_values('document_frequency', ascending=False) # 場合によってはコメントアウト

,terms,document_frequency,max_term_frequency,sumof_term_frequency
7181,貫通孔,1937,10,2738
4825,支持部材,1129,18,1445
7484,連結部材,1065,11,1445
2632,制御装置,966,93,1306
2350,免震装置,905,24,1155
...,...,...,...,...
4933,施工箇所,5,1,5
1901,丸穴,5,1,5
6792,自動ラック倉庫,5,2,6
4937,施工範囲,5,1,5


In [24]:
terms_df

,0
0,あと施工アンカー
1,かご鉄筋
2,かさ歯車
3,かんぬき梁
4,かんぬき部材
...,...
8242,ｓｒｃ造
8243,ｔｂｍ本体
8244,ｔｍｄ機構
8245,ｗｅｂサーバ


In [22]:
#単語:文書内単語頻度 の辞書の列つくって、pandasにくっつける
def makeTermsTfDict(terms, vec):
    dic = {key: val for key, val in zip(terms, vec)  if val } #dfが1より大きいtermの辞書つくる
#     dic = {key: val for key, val in zip(terms, vec)  }
    dic2 = dict(sorted(dic.items(), key=lambda x:x[1],reverse=True)) 
    elm = ''
    text = ''
    for k, v in dic2.items():
        elm += f'{k}|'
    text = elm[:-1]
    return text

df['terms'] = pd.Series([makeTermsTfDict(terms, vec) for vec in [c_matrix.getrow(i).toarray()[0] for i in range(c_matrix.shape[0])]]) #countされたベクトルの各行を関数に入れていってシリーズに

In [10]:
df.dtypes

出願年         object
出願人・権利者名    object
IPC         object
Ｆターム        object
要約          object
詳細な説明       object
all         object
terms       object
dtype: object

In [11]:
index = [i for i in range(len(df))]
df['id'] = index

In [12]:
l = []
for company in df['出願人・権利者名']:
    c = company.replace(',', '|')
    l.append(c)

f = []
for f_term in df['Ｆターム']:
    if type(f_term) == str:
        t = f_term.replace(',', '|')
        f.append(t)
    else:
        f.append(np.nan)
                
ip = []
for ipc in df['IPC']:
    i = ipc.replace(',', '|')
    ip.append(i)

df['year'] = df['出願年']

df['company'] = l
df['f_term'] = f
df['IPC'] = ip
new_df = df.reindex(['id', 'company', 'IPC', 'f_term', 'terms', 'year'], axis='columns')

In [13]:
new_df.head(10)

,id,company,IPC,f_term,terms,year
0,0,佐藤工業株式会社,B65D,3E060AA03|3E060AB05|3E060BC02|3E060CB04|3E060C...,仕切り壁,2001
1,1,寄神建設株式会社|東亜建設工業株式会社|社団法人日本海上起重技術協会,B66C,3F004CC01|3F004CD08,玉掛けワイヤ,2001
2,2,株式会社大林組,E01D,2D059AA03|2D059CC04,,2001
3,3,株式会社大林組,E02B|B65G,3F040BA01|3F040CA04|3F040EA03,コンクリートプラント|回転駆動機構|螺旋羽根,2001
4,4,株式会社大林組,E02B,NaN,コンクリートプラント|回転駆動機構|螺旋羽根,2001
5,5,株式会社大林組,E02B|B65G,3F040BA01|3F040EA03|3F040FA03|3F040FA05,コンクリートプラント|回転駆動機構|螺旋羽根,2001
6,6,株式会社大林組,E02B|B65G,3F011AA02|3F011BA01|3F011BC07,コンクリートプラント,2001
7,7,大阪瓦斯株式会社|株式会社竹中工務店|株式会社竹中土木|三谷セキサン株式会社,E02D,2D041AA02|2D041BA13|2D041BA19|2D041CA03|2D041C...,コンクリート杭|鋼管杭|ｐｃ鋼棒|連結継手|貫通孔|ねじ孔|合部材|掘削ヘッド|杭穴|補強部...,2001
8,8,株式会社長谷工コーポレーション,E04H,NaN,,2001
9,9,株式会社長谷工コーポレーション,E04H,NaN,,2001


In [14]:
new_df.isnull()

,id,company,IPC,f_term,terms,year
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,True,False,False
...,...,...,...,...,...,...
44559,False,False,False,False,False,False
44560,False,False,False,False,False,False
44561,False,False,False,False,False,False
44562,False,False,False,False,False,False


In [15]:
with open('/app/patent/df_with_term.pkl', 'wb') as f:
    pickle.dump(new_df, f)

In [23]:
df['terms'] 

0                                           仕切り壁
1                                         玉掛けワイヤ
2                                               
3                         コンクリートプラント|回転駆動機構|螺旋羽根
4                         コンクリートプラント|回転駆動機構|螺旋羽根
                          ...                   
44559    補強部材|壁高欄|ボルト孔|収縮モルタル|貫通孔|コンクリート床板|固定ナット
44560                    エネルギー吸収部材|壁部材|継手鉄筋|弾性部材
44561                                       機械学習
44562                                       鉄イオン
44563       ハイブリッド梁|バンドプレート|プレート部材|保持部材|支持部材|梁主筋
Name: terms, Length: 44564, dtype: object

In [35]:
new_term = []
for sentence in df['all']:
    term_list = []
    for row in terms_df.itertuples():
        if row[1] in sentence:
            term_list.append(row[1])
    new_term.append(term_list)

In [43]:
l = []
for elm in df['sample']:
    u = []
    for row in elm:
        u.append(row[1])
    l.append(u)

In [45]:
df['term_all'] = l

In [47]:
l = []
for tl in df['term_all']:
    s = ''
    for t in tl:
        s += f'{t}|'
    txt = s[:-1]
    l.append(txt)

In [48]:
df['term_all'] = l

In [49]:
df

,出願年,出願人・権利者名,IPC,Ｆターム,要約,詳細な説明,all,terms,id,year,company,f_term,sample,term_all
0,2001,佐藤工業株式会社,B65D,"3E060AA03,3E060AB05,3E060BC02,3E060CB04,3E060C...",【要約】【課題】 強度が劣る原紙を用いたとしても箱の強度が維持される外装用段ボール箱を得んと...,【考案の詳細な説明】 【０００１】 【考案の属する技術分野】 本考案は、袋菓子または詰...,【要約】【課題】 強度が劣る原紙を用いたとしても箱の強度が維持される外装用段ボール箱を得んと...,仕切り壁,0,2001,佐藤工業株式会社,3E060AA03|3E060AB05|3E060BC02|3E060CB04|3E060C...,"[(1636, 一実施例), (1639, 一対), (1746, 上縁), (1984, ...",一実施例|一対|上縁|仕切り壁|仕切り板|仕切板|加圧|圧縮強度|実施例|技術分野|箱本体|連設
1,2001,"寄神建設株式会社,東亜建設工業株式会社,社団法人日本海上起重技術協会",B66C,"3F004CC01,3F004CD08",【要約】【課題】 重量物を吊り下げて設置を行う際の、重量物の開放を遠隔操作で容易かつ安全に行...,【考案の詳細な説明】【０００１】【考案の属する技術分野】 本考案は、クレーン等によりコンクリ...,【要約】【課題】 重量物を吊り下げて設置を行う際の、重量物の開放を遠隔操作で容易かつ安全に行...,玉掛けワイヤ,1,2001,寄神建設株式会社|東亜建設工業株式会社|社団法人日本海上起重技術協会,3F004CC01|3F004CD08,"[(1762, 下げワイヤ), (3031, 回動), (3783, 実施形態), (422...",下げワイヤ|回動|実施形態|技術分野|玉掛けワイヤ|遠隔操作
2,2001,株式会社大林組,E01D,"2D059AA03,2D059CC04",【要約】【目的】 作業足場や型枠の組立，解体が不要になる橋脚柱の構築工法の提供。【構成】 構...,【発明の詳細な説明】【０００１】【産業上の利用分野】この発明は、橋脚柱の構築工法および橋脚柱...,【要約】【目的】 作業足場や型枠の組立，解体が不要になる橋脚柱の構築工法の提供。【構成】 構...,,2,2001,株式会社大林組,2D059AA03|2D059CC04,"[(469, コンクリート打設), (474, コンクリート打設工), (489, コンクリ...",コンクリート打設|コンクリート打設工|コンクリート構造|一実施例|作業用足場|作業足場|係止...
3,2001,株式会社大林組,E02B|B65G,"3F040BA01,3F040CA04,3F040EA03",【要約】【課題】 材料分離と運搬費用の低減。【解決手段】 シュート式打設装置１２は、円筒状の...,【発明の詳細な説明】【０００１】【発明の属する技術分野】この発明は、コンクリートの運搬装置に...,【要約】【課題】 材料分離と運搬費用の低減。【解決手段】 シュート式打設装置１２は、円筒状の...,コンクリートプラント|回転駆動機構|螺旋羽根,3,2001,株式会社大林組,3F040BA01|3F040CA04|3F040EA03,"[(405, ケーブルクレーン), (438, コンクリートダム), (440, コンクリー...",ケーブルクレーン|コンクリートダム|コンクリートバケット|コンクリートプラント|コンクリート...
4,2001,株式会社大林組,E02B,NaN,【要約】【課題】 材料分離と運搬費用の低減。【解決手段】 シュート式打設装置１２は、円筒状の...,【発明の詳細な説明】【０００１】【発明の属する技術分野】この発明は、コンクリートの運搬方法に...,【要約】【課題】 材料分離と運搬費用の低減。【解決手段】 シュート式打設装置１２は、円筒状の...,コンクリートプラント|回転駆動機構|螺旋羽根,4,2001,株式会社大林組,NaN,"[(405, ケーブルクレーン), (438, コンクリートダム), (440, コンクリー...",ケーブルクレーン|コンクリートダム|コンクリートバケット|コンクリートプラント|コンクリート...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44559,2019,"三井住友建設株式会社,ファイベックス株式会社",E01D,"2D059AA21,2D059GG55,2D059GG40,2D059GG02",(57)【要約】【課題】コンクリートからなる壁高欄の下端部における内側及び外側の両面に引張部...,【発明の詳細な説明】【技術分野】【０００１】 本発明は、コンクリート床版を有する橋梁、高架橋...,(57)【要約】【課題】コンクリートからなる壁高欄の下端部における内側及び外側の両面に引張部...,補強部材|壁高欄|ボルト孔|収縮モルタル|貫通孔|コンクリート床板|固定ナット,44559,2019,三井住友建設株式会社|ファイベックス株式会社,2D059AA21|2D059GG55|2D059GG40|2D059GG02,"[(58, アラミド繊維), (176, エポキシ樹脂), (327, ガラス繊維), (4...",アラミド繊維|エポキシ樹脂|ガラス繊維|コンクリート床|コンクリート床板|コンクリート構造|...
44560,2019,三井住友建設株式会社,E04H|F16F|E04B,"2E139AA01,2E139AB03,2E139BD24,2E139BD23,2E139B...",(57)【要約】【課題】壁のＰＣａ化に伴う問題を解消でき、且つ期待される減衰力を地震時に確実...,【発明の詳細な説明】【技術分野】【０００１】 本開示は、建物の制振構造及びその構築方法に関し...,(57)【要約】【課題】壁のＰＣａ化に伴う問題を解消でき、且つ期待される減衰力を地震時に確実...,エネルギー吸収部材|壁部材|継手鉄筋|弾性部材,44560,2019,三井住友建設株式会社,2E139AA01|2E139AB03|2E139BD24|2E139BD23|2E139B...,"[(58, アラミド繊維), (170, エネルギー吸収), (172, エネルギー吸収装置...",アラミド繊維|エネルギー吸収|エネルギー吸収装置|エネルギー吸収部材|ガラス繊維|セメント系...
44561,2019,清水建設株式会社,G01S,"5J062AA09,5J062CC07,5J062EE04,5J062EE01,5J062DD23",(57)【要約】【課題】測位アルゴリズムにおける最良の設定パラメータを迅速かつ確度良く決定す...,【発明の詳細な説明】【技術分野】【０００１】 本発明は、測位アルゴリズムにおける最良の設定パ...,(57)【要約】【課題】測位アルゴリズムにおける最良の設定パラメータを迅速かつ確度良く決定す...,機械学習,44561,2019,清水建設株式会社,5J062AA09|5J062CC07|5J062EE04|5J062EE01|5J062DD23,"[(921, データ取得), (1974, 人工衛星), (4226, 技術分野), (48...",データ取得|人工衛星|技術分野|教師データ|機械学習|添付図面|特開|登録商標
44562,2019,株式会社フジタ,C05B|C05G,"4H061AA01,4H061BB29,4H061EE16,4H061LL26,4H061L...",(57)【要約】【課題】作物に対する肥効成分を長期間利用可能な肥料を提供すること。【解決手段...,【発明の詳細な説明】【技術分野】【０００１】 本発明は肥料に関する。特に、本発明はリン及び鉄...,(57)【要約】【課題】作物に対する肥効成分を長期間利用可能な肥料を提供すること。【解決手段...,鉄イオン,44562,2019,株式会社フジタ,4H061AA01|4H061BB29|4H061EE16|4H061LL26|4H061L...,"[(1042, バイオマス), (1637, 一実施形態), (2127, 作用効果), (...",バイオマス|一実施形態|作用効果|加圧|加熱装置|圧力容器|孔径|実施例|実施形態|技術分野...


In [50]:
new_df = df.reindex(['id', 'company', 'IPC', 'f_term', 'term_all', 'year'], axis='columns')

In [51]:
new_df

,id,company,IPC,f_term,term_all,year
0,0,佐藤工業株式会社,B65D,3E060AA03|3E060AB05|3E060BC02|3E060CB04|3E060C...,一実施例|一対|上縁|仕切り壁|仕切り板|仕切板|加圧|圧縮強度|実施例|技術分野|箱本体|連設,2001
1,1,寄神建設株式会社|東亜建設工業株式会社|社団法人日本海上起重技術協会,B66C,3F004CC01|3F004CD08,下げワイヤ|回動|実施形態|技術分野|玉掛けワイヤ|遠隔操作,2001
2,2,株式会社大林組,E01D,2D059AA03|2D059CC04,コンクリート打設|コンクリート打設工|コンクリート構造|一実施例|作業用足場|作業足場|係止...,2001
3,3,株式会社大林組,E02B|B65G,3F040BA01|3F040CA04|3F040EA03,ケーブルクレーン|コンクリートダム|コンクリートバケット|コンクリートプラント|コンクリート...,2001
4,4,株式会社大林組,E02B,NaN,ケーブルクレーン|コンクリートダム|コンクリートバケット|コンクリートプラント|コンクリート...,2001
...,...,...,...,...,...,...
44559,44559,三井住友建設株式会社|ファイベックス株式会社,E01D,2D059AA21|2D059GG55|2D059GG40|2D059GG02,アラミド繊維|エポキシ樹脂|ガラス繊維|コンクリート床|コンクリート床板|コンクリート構造|...,2019
44560,44560,三井住友建設株式会社,E04H|F16F|E04B,2E139AA01|2E139AB03|2E139BD24|2E139BD23|2E139B...,アラミド繊維|エネルギー吸収|エネルギー吸収装置|エネルギー吸収部材|ガラス繊維|セメント系...,2019
44561,44561,清水建設株式会社,G01S,5J062AA09|5J062CC07|5J062EE04|5J062EE01|5J062DD23,データ取得|人工衛星|技術分野|教師データ|機械学習|添付図面|特開|登録商標,2019
44562,44562,株式会社フジタ,C05B|C05G,4H061AA01|4H061BB29|4H061EE16|4H061LL26|4H061L...,バイオマス|一実施形態|作用効果|加圧|加熱装置|圧力容器|孔径|実施例|実施形態|技術分野...,2019


In [52]:
with open('/app/patent/df_with_term.pkl', 'wb') as f:
    pickle.dump(new_df, f)